In [38]:
import cv2
import numpy as np
tensorflowNet = cv2.dnn.readNet('simple_frozen_graph.pb', 'simple_frozen_graph.pbtxt.txt')
CLASSES = ['Blast', 'FalseSmut', 'RiceBug', 'SheathRot', 'StemBorer', 'unrelated_image']
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))
img = cv2.imread('E:/Official purpose/2021/Sprint - 26/Crop images/ricebug_download3.jpg')
rows, cols, channels = img.shape
tensorflowNet.setInput(cv2.dnn.blobFromImage(img, size=(299, 299), swapRB=True, crop=False))
networkOutput = tensorflowNet.forward()


for detection in range(int(networkOutput[0,0])):
    score = float(detection[2])
    if score > 0.2:
        class_id = detection[1]
        class_name = CLASSES[int(class_id)-1]
        print(class_name)
        color = COLORS[int(class_id)]
        left = detection[3] * cols
        top = detection[4] * rows
        right = detection[5] * cols
        bottom = detection[6] * rows
        cv2.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), color, thickness=2)
        cv2.putText(image, class_name, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
cv2.imshow('Image', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [38]:
import numpy as np
import tensorflow as tf
import cv2 as cv


with tf.io.gfile.GFile('simple_frozen_graph.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

    
with tf.compat.v1.Session() as sess:
    sess.graph.as_default()
    tf.compat.v1.import_graph_def(graph_def, name='')
    img = cv.imread('E:/Official purpose/2021/Sprint - 26/Crop images/ricebug_download3.jpg')
    rows = img.shape[0]
    cols = img.shape[1]
    inp = cv.resize(img, (300, 300))
    inp = inp[:, :, [2, 1, 0]]  
    out = sess.run([
                    sess.graph.get_tensor_by_name('Identity:0'),
                    sess.graph.get_tensor_by_name('Identity:0'),
                    sess.graph.get_tensor_by_name('Identity:0'),
                    sess.graph.get_tensor_by_name('Identity:0')
                    ],
                    feed_dict={'Identity:0': 
                                inp.reshape(1, inp.shape[0], inp.shape[1], 3)
                               #inp.reshape(45000, 6)
                              })
    num_detections = int(out[0][0])
    for i in range(num_detections):
        classId = int(out[3][0][i])
        score = float(out[1][0][i])
        bbox = [float(v) for v in out[2][0][i]]
        if score > 0.3:
            x = bbox[1] * cols
            y = bbox[0] * rows
            right = bbox[3] * cols
            bottom = bbox[2] * rows
            cv.rectangle(img, (int(x), int(y)), (int(right), int(bottom)), (125, 255, 51), thickness=2)
cv.imshow('TensorFlow Inception-SSD', img)
cv.waitKey()

ValueError: Cannot feed value of shape (1, 300, 300, 3) for Tensor 'Identity:0', which has shape '(None, 6)'